In [1]:
import pandas as pd
from tqdm import tqdm_notebook

In [2]:
essalud = pd.ExcelFile('data/input/Essalud en Cifras - Definitivo Diciembre 2015.xls')

In [3]:
sheet_names = essalud.sheet_names[1:10]

In [4]:
consulta_ext = ['CONSULTANTES ASEGURADOS','CONSULTANTES DEL SERVICIO','CONSULTANTES NUEVOS',
                'CONSULTAS CAI','CONSULTORIO FUNCIONAL','CONSULTORIOS FISICOS',
                'HORAS MEDICAS EFECTIVAS','HORAS MEDICAS PROGRAMADAS','MEDICOS',
                'TOTAL CONSULTANTES','TOTAL CONSULTAS', 'CONSULTAS MEDICINA COMPLEMENTARIA']

hospitalizacion = ['CAMAS HOSPITALARIAS','DEFUNCIONES ANTES 48 HORAS',
                   'DEFUNCIONES DESPUES 48 HORAS','DIAS CAMA DISPONIBLES','EGRESOS',
                   'ESTANCIA','PACIENTES DIA','REINGRESOS','PACIENTE DIA UNIDAD CORONARIA']

centr_quirurgico = ['CIRUGIA MAYOR ALTA COMPLEJIDAD','CIRUGIA MAYOR BAJA COMPLEJIDAD',
                    'CIRUGIA MAYOR MEDIANA COMPLEJIDAD','CIRUGIA MENOR ALTO REQUERIMIENTO',
                    'CIRUGIA MENOR BAJO REQUERIMIENTO',
                    'HORAS PROGRAMADAS INTERVENCION QUIRURGIC','HORAS USO DE SALA',
                    'INTERVENCION QUIRURGICA PROGRAMADAS',
                    'INTERVENCIONES QUIRURGICAS SUSPENDIDAS','SALAS QUIRURGICAS FUNCIONALES']

centr_obstetrico = ['CESAREAS','NACIDOS VIVOS','NACIMIENTOS','PARTOS CON COMPLICACION',
                    'PARTOS SIN COMPLICACION','N° SALAS DE PARTOS',
                    'PACIENTE DIA - PARTOS COMPLICADOS',
                    'PACIENTE DIA - PARTOS SIN COMPLICACIÓN']

emergencia = ['ATENCIONES DE EMERGENCIA I','ATENCIONES DE EMERGENCIA II',
              'ATENCIONES DE EMERGENCIA III','ATENCIONES DE EMERGENCIA IV',
              'EGRESO PACIENTE <=24 HORAS','EGRESO PACIENTE > 24 HORAS']

odontologia = ['CONSULTANTES ODONTOLOGICOS','CONSULTORIOS FISICOS ODONTOLOGIA',
               'CONSULTORIOS FUNCIONALES ODONTOLOGIA','HORAS ODONTOLOGICAS EFECTIVAS',
               'HORAS ODONTOLOGICAS PROGRAMADAS','ODONTOLOGOS','SESIONES ODONTOLOGICAS',
               'TRABAJOS ODONTOLOGICOS']

act_interm = ['ANALISIS LABORATORIO CE','EXAMENES DE LABORATORIO','EXAMENES RADIOLOGICOS',
              'EXAMENES RADIOLOGICOS CE','PLACAS RADIOLOGICAS','PLACAS RADIOLOGICAS CE',
              'RECETAS DESPACHADAS','RECETAS DESPACHADAS CE']

at_primero = ['MENORES 1 AÑO TERCERA DOSIS PENTAVALENTE',
              'MUESTRAS PROCESADAS X PAP CERVICO VAGINA',
              'PRIMERA CONSULTA EXTERNA M. PREVENTIVA','CRED ADOLESCENTES','CRED NIÑOS',
              'MAMOGRAFIAS PREVENTIVAS 50 A 74 (RESULTA']

at_sec = ['ACTIVIDADES EDUCACION GRUPAL TS','ATENCIONES PSICOLOGICAS','CONTROL DE ENFERMERIA',
          'CONTROL DE NUTRICION','CONTROL DE OBSTETRICIA','CONTROL DE TRABAJO SOCIAL',
          'EDUCACION GRUPAL DE ENFERMERIA','EDUCACION GRUPAL DE NUTRICION',
          'EDUCACION GRUPAL DE OBSTETRICIA','PSICOPROFILAXIS DE NUTRICION',
          'PSICOPROFILAXIS DE OBSTETRICIA','PSICOPROFILAXIS ENFERMERIA',
          'VISITAS DOMICILIARIAS','VISITAS DOMICILIARIAS DE NUTRICION',
          'VISITAS DOMICILIARIAS DE OBSTETRICIA','VISITAS DOMICILIARIAS TS',
          'PERSONAL DE ENFERMERIA','PERSONAL DE OBSTETRICIA','PSICOPROFILAXIS DE PSICOLOGIA',
          'VISITAS DOMICILIARIAS DE PSICOLOGIA','PSICOPROFILAXIS TS']



variables_lists = [consulta_ext, hospitalizacion, centr_quirurgico, centr_obstetrico,
                   emergencia, odontologia, act_interm, at_primero, at_sec]

red_asist = ['ALMENARA','AMAZONAS','ANCASH','APURIMAC','AREQUIPA','AYACUCHO','CAJAMARCA',
             'CENTRO NACIONAL SALUD RENAL','CUSCO','HUANCAVELICA','HUANUCO','ICA','INCOR',
             'JULIACA','JUNIN','LA LIBERTAD','LAMBAYEQUE','LORETO','MADRE DE DIOS',
             'MOQUEGUA','MOYOBAMBA','PASCO','PIURA','PUNO','REBAGLIATI','SABOGAL',
             'TACNA','TARAPOTO','TUMBES','UCAYALI','HUARAZ']


In [5]:
n_vars = 0
for list_ in variables_lists:
    n_vars += len(list_)

In [6]:
n_vars

88

In [7]:
year = '2015'

dfs = []

for i, sheet_name in enumerate(sheet_names):
    df = essalud.parse(sheet_name=sheet_name, skiprows=9)
    df = df.iloc[:,1:]
    print('Sheet: {}'.format(sheet_name))
    for j in tqdm_notebook(range(len(variables_lists[i])), total=len(variables_lists[i])):
    
        if j < len(variables_lists[i])-1:
            init_ix = df[df['VARIABLE / RED ASISTENCIAL / CENTRO ASISTENCIAL'] == variables_lists[i][j]].index.values[0] + 1
            final_ix = df[df['VARIABLE / RED ASISTENCIAL / CENTRO ASISTENCIAL'] == variables_lists[i][j+1]].index.values[0] - 1
            df_var1 = df.loc[init_ix:final_ix,:]
        else:
            init_ix = df[df['VARIABLE / RED ASISTENCIAL / CENTRO ASISTENCIAL'] == variables_lists[i][j]].index.values[0] + 1
            df_var1 = df.loc[init_ix:init_ix+431,:]

        red_asist_filter = [df_var1['VARIABLE / RED ASISTENCIAL / CENTRO ASISTENCIAL'] != red_asist_val for red_asist_val in red_asist]
        
        # Generate Boleean Series to filter RED_ASIST VARS
        for k in range(1, len(red_asist_filter)):
            if k == 1:
                temp = red_asist_filter[k] & red_asist_filter[k-1]
            else:
                temp = red_asist_filter[k] & temp

        not_redasist = temp
                      
        df_var1 = df_var1[not_redasist]
        df_var1 = df_var1.melt(id_vars=df_var1.columns[0], value_vars=df_var1.columns[1:])
        df_var1 = df_var1.rename(index=str, 
                                 columns={"VARIABLE / RED ASISTENCIAL / CENTRO ASISTENCIAL": "centro_asist",
                                          "variable": "month",
                                          "value": variables_lists[i][j]})
        df_var1['month'] = df_var1['month'].replace(to_replace=df_var1['month'].unique(),
                                                    value=['TOTAL','01','02','03','04','05','06','07','08','09','10', '11', '12'])
        df_var1['centro_asist'] = df_var1['centro_asist'].str.extract(pat='([A-Z .]+)')
        df_var1.index = df_var1['centro_asist'] + '_' + 'per' + '_' + year + '-' + df_var1['month']
        df_var1.drop(labels=['centro_asist','month'], axis=1, inplace=True)
        df_var1.dropna(inplace=True)
        df_var1.sort_index(inplace=True)
        
        dfs.append(df_var1.T)


Sheet: CONSULTA EXTERNA



Sheet: HOSPITALIZACION



Sheet: CENTRO QUIRURGICO



Sheet: CENTRO OBSTETRICO



Sheet: EMERGENCIA



Sheet: ODONTOLOGIA



Sheet: ACTIVIDADES INTERMEDIAS



Sheet: ATENCION PRIMARIA



Sheet: ATENCION NOMEDICA


In [8]:
final_df = dfs[0]

for i in range(1,len(dfs)):
    final_df = final_df.append(dfs[i])

In [9]:
final_df = final_df.T

In [10]:
final_df.head()

,CONSULTANTES ASEGURADOS,CONSULTANTES DEL SERVICIO,CONSULTANTES NUEVOS,CONSULTAS CAI,CONSULTORIO FUNCIONAL,CONSULTORIOS FISICOS,HORAS MEDICAS EFECTIVAS,HORAS MEDICAS PROGRAMADAS,MEDICOS,TOTAL CONSULTANTES,...,PSICOPROFILAXIS ENFERMERIA,VISITAS DOMICILIARIAS,VISITAS DOMICILIARIAS DE NUTRICION,VISITAS DOMICILIARIAS DE OBSTETRICIA,VISITAS DOMICILIARIAS TS,PERSONAL DE ENFERMERIA,PERSONAL DE OBSTETRICIA,PSICOPROFILAXIS DE PSICOLOGIA,VISITAS DOMICILIARIAS DE PSICOLOGIA,PSICOPROFILAXIS TS
C.E. CENTRO NACIONAL DE SALUD RENAL _per_2015-01,44,44,0,0,1,1,12,12,3,44,...,0,22,0,0,17,57,0,0,0,0
C.E. CENTRO NACIONAL DE SALUD RENAL _per_2015-02,0,0,0,0,1,1,12,12,3,0,...,0,25,0,0,21,51,0,0,0,0
C.E. CENTRO NACIONAL DE SALUD RENAL _per_2015-03,2,2,1,0,1,1,12,12,2,2,...,0,23,0,0,19,52,0,0,0,0
C.E. CENTRO NACIONAL DE SALUD RENAL _per_2015-04,1,1,1,0,1,1,12,12,2,1,...,0,25,0,0,21,54,0,0,0,0
C.E. CENTRO NACIONAL DE SALUD RENAL _per_2015-05,0,0,0,0,2,1,12,12,3,0,...,0,24,0,0,20,56,0,0,0,0


In [11]:
final_df.to_csv('data/output/essalud_data_2015.csv')